## ELECTRA

In [ ]:
import torch
import numpy as np
import pandas as pd
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import TrainingArguments, Trainer
from torch.utils.data import Dataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#R2
# train_data=pd.read_csv(r"C:\Users\MSI-PC\Desktop\Dataset\R2_train.csv",header=0,names=['category','text'])
# test_data=pd.read_csv(r"C:\Users\MSI-PC\Desktop\Dataset\R2_test.csv",header=0,names=['category','text'])
# train_data['category'] = train_data['category'].map({'acq':0,'earn':1})
# test_data['category'] = test_data['category'].map({'acq':0,'earn':1})
#yahoo
train_data=pd.read_csv(r"C:\Users\MSI-PC\Desktop\Dataset\yahoo_train.csv",header=0,names=['category','text'])
test_data=pd.read_csv(r"C:\Users\MSI-PC\Desktop\Dataset\yahoo_test.csv",header=0,names=['category','text'])
train_data['category'] = train_data['category']-1
test_data['category'] = test_data['category']-1
train_data.dropna(inplace=True)
test_data.dropna(inplace=True)
train_data.head()

,category,text
0,0,Good question.
1,0,for religious or philosophical matter yes. no...
2,0,My stomach growled so loud once in Church that...
3,0,"The same reasons women do. \n\nthe ""game""...i..."
4,0,That is my hope. Unless we examine the mistak...


In [ ]:
print(len(train_data))
print(len(test_data))
num_labels = len(test_data['category'].unique())
num_labels

3928
984


10

In [ ]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")

loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/vocab.txt from cache at C:\Users\MSI-PC/.cache\huggingface\transformers\ece45ade3e01224cf31fed8e183b306d17b84e8abd415363474cfe72274f7814.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/tokenizer_config.json from cache at C:\Users\MSI-PC/.cache\huggingface\transformers\8b3aea9f7242b3d19268df5b1bfed8f66e08671a72ac0809ada08e5ef1adc592.19eda9a6da5fb0e52a45200c95876729561dde16a69b9116953af6edca1d1e92
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/tokenizer.json from cache at C:\Users\MSI-PC/.cache\huggingface\transformers\92992b3

In [ ]:
def encode_data(dataframe,tokenizer,max_seq_length=64):
    inputs = list(dataframe['text'])
    encoded = tokenizer(inputs,max_length=max_seq_length,truncation=True,padding="max_length",return_tensors="pt")
    return encoded

def extract_labels(dataframe):
    return list(dataframe['category'])

def model_init():
    model = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator",num_labels=num_labels)
    return model

def compute_metrics(eval_pred):
    labels = eval_pred.label_ids
    preds = eval_pred.predictions.argmax(-1)
    from sklearn.metrics import accuracy_score, normalized_mutual_info_score, adjusted_rand_score
    accurcay = accuracy_score(labels,preds)
    NMI = normalized_mutual_info_score(labels,preds)
    ARI = adjusted_rand_score(labels,preds)
    return {'eval_accuracy':accurcay,'eval_NMI':NMI,'eval_ARI':ARI}

In [ ]:
class CreateDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_seq_length=64):
        self.encoded_data = encode_data(dataframe,tokenizer,max_seq_length)
        self.label_list = extract_labels(dataframe)

    def __len__(self): 
        return len(self.label_list)

    def __getitem__(self, i):
        item_i = {}
        item_i['input_ids'] = self.encoded_data['input_ids'][i]
        item_i['attention_mask'] = self.encoded_data['attention_mask'][i]
        item_i['labels'] = self.label_list[i]
        
        return item_i

In [ ]:
#split train_data into training and val data
training_data = train_data.sample(frac=0.8, random_state=8521)
val_data = train_data.drop(training_data.index)

train_data_electra = CreateDataset(training_data, tokenizer)
val_data_electra = CreateDataset(val_data, tokenizer)
test_data_electra = CreateDataset(test_data, tokenizer)
train_data_electra[0]

{'input_ids': tensor([  101,  2224,  1061,  1027, 25630,  1009,  1038,  2000,  2424,  2115,
          8522,  1012,  1049,  2003,  1011,  1017,  1010,  1998,  2017,  2031,
          2019,  1060,  1998,  1037,  1061,  1010,  2061,  2017,  2064,  2424,
          1038,  1012,  1012,  1012,  1032,  1050,  1032,  6396,  1027, 25630,
          1009,  1038,  1032,  1050,  2629,  1027,  1011,  1017,  1008,  1016,
          1009,  1038,  1032,  1050,  2629,  1027,  1011,  1020,  1009,  1038,
          1032,  1050,  2497,   102]),
 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 'labels': 3}

In [ ]:
training_args = TrainingArguments(
    output_dir="C:/Users/MSI-PC/Desktop/DS-GA 1012 (NLU)/Term Project/models_yahoo",
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=100, 
    per_device_eval_batch_size=100,
    num_train_epochs=2,
    logging_steps=2, 
    logging_first_step=True,
    save_steps=64, 
    evaluation_strategy = "epoch", # evaluate at the end of every epoch
    logging_dir="C:/Users/MSI-PC/Desktop/DS-GA 1012 (NLU)/Term Project/logs",
    learning_rate=1e-5, #config
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(args = training_args,
                  train_dataset=train_data_electra,
                  eval_dataset=val_data_electra,
                  tokenizer=tokenizer,
                  model_init = model_init,
                  compute_metrics = compute_metrics,)

loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at C:\Users\MSI-PC/.cache\huggingface\transformers\ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LA

In [ ]:
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.suggest.basic_variant import BasicVariantGenerator

In [ ]:
tune_config = {"learning_rate": tune.uniform(1e-5, 5e-5)} 

best_results = trainer.hyperparameter_search(
    hp_space = lambda _:tune_config,
    backend = 'ray',
    compute_objective = lambda metrics: metrics["eval_ARI"],
    mode = 'max',
    search_alg = BasicVariantGenerator(),
    n_trials=3, 
)

print(best_results)

No `resources_per_trial` arg was passed into `hyperparameter_search`. Setting it to a default value of 1 CPU for each trial.
2022-05-12 20:51:49,870	WARNING tune.py:580 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.


== Status ==
Current time: 2022-05-12 20:51:49 (running for 00:00:00.13)
Memory usage on this node: 13.5/31.8 GiB
Using FIFO scheduling algorithm.
Resources requested: 0/16 CPUs, 0/1 GPUs, 0.0/11.87 GiB heap, 0.0/5.94 GiB objects
Result logdir: C:\Users\MSI-PC\ray_results\_objective_2022-05-12_20-51-49
Number of trials: 3/3 (3 PENDING)
+------------------------+----------+-------+-----------------+
| Trial name             | status   | loc   |   learning_rate |
|------------------------+----------+-------+-----------------|
| _objective_df4f2_00000 | PENDING  |       |     2.49816e-05 |
| _objective_df4f2_00001 | PENDING  |       |     4.80286e-05 |
| _objective_df4f2_00002 | PENDING  |       |     3.92798e-05 |
+------------------------+----------+-------+-----------------+




 pid=21808) 2022-05-12 20:51:51.578440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
 pid=21808) 2022-05-12 20:51:51.578482: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
 pid=18228) 2022-05-12 20:51:51.578505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
 pid=18228) 2022-05-12 20:51:51.578537: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
 pid=3448) 2022-05-12 20:51:51.578936: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
 pid=3448) 2022-05-12 20:51:51.578969: I tensorflow/stream_executor/cuda/cudart_stub.cc:

== Status ==
Current time: 2022-05-12 20:51:55 (running for 00:00:05.25)
Memory usage on this node: 15.1/31.8 GiB
Using FIFO scheduling algorithm.
Resources requested: 3.0/16 CPUs, 0/1 GPUs, 0.0/11.87 GiB heap, 0.0/5.94 GiB objects
Result logdir: C:\Users\MSI-PC\ray_results\_objective_2022-05-12_20-51-49
Number of trials: 3/3 (3 RUNNING)
+------------------------+----------+-----------------+-----------------+
| Trial name             | status   | loc             |   learning_rate |
|------------------------+----------+-----------------+-----------------|
| _objective_df4f2_00000 | RUNNING  | 127.0.0.1:3448  |     2.49816e-05 |
| _objective_df4f2_00001 | RUNNING  | 127.0.0.1:21808 |     4.80286e-05 |
| _objective_df4f2_00002 | RUNNING  | 127.0.0.1:18228 |     3.92798e-05 |
+------------------------+----------+-----------------+-----------------+


== Status ==
Current time: 2022-05-12 20:52:00 (running for 00:00:10.32)
Memory usage on this node: 16.4/31.8 GiB
Using FIFO scheduling algo

                                              
  2%|▏         | 1/64 [00:06<07:15,  6.91s/it]


...

Result for _objective_df4f2_00001:
  date: 2022-05-12_20-59-40
  done: false
  epoch: 2.0
  eval_ARI: 0.06685069509931021
  eval_NMI: 0.1367551557863934
  eval_accuracy: 0.272264631043257
  eval_loss: 2.2561981678009033
  eval_runtime: 19.7182
  eval_samples_per_second: 39.862
  eval_steps_per_second: 0.406
  experiment_id: 2a2a64a433f04804921999771ab2620d
  hostname: XR
  iterations_since_restore: 2
  node_ip: 127.0.0.1
  objective: 0.06685069509931021
  pid: 21808
  time_since_restore: 466.6441249847412
  time_this_iter_s: 233.13328051567078
  time_total_s: 466.6441249847412
  timestamp: 1652403580
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: df4f2_00001
  
Result for _objective_df4f2_00001:
  date: 2022-05-12_20-59-40
  done: true
  epoch: 2.0
  eval_ARI: 0.06685069509931021
  eval_NMI: 0.1367551557863934
  eval_accuracy: 0.272264631043257
  eval_loss: 2.2561981678009033
  eval_runtime: 19.7182
  eval_samples_per_second: 39.862
  eval_steps_per_second: 0.406
  ex

 pid=21808) 

100%|██████████| 64/64 [07:45<00:00,  7.28s/it]


Result for _objective_df4f2_00000:
  date: 2022-05-12_20-59-40
  done: false
  epoch: 2.0
  eval_ARI: 0.012606357983127578
  eval_NMI: 0.062285683568066096
  eval_accuracy: 0.15903307888040713
  eval_loss: 2.287787437438965
  eval_runtime: 19.7602
  eval_samples_per_second: 39.777
  eval_steps_per_second: 0.405
  experiment_id: 585b7a9941b44caba7dee0c7fac062c2
  hostname: XR
  iterations_since_restore: 2
  node_ip: 127.0.0.1
  objective: 0.012606357983127578
  pid: 3448
  time_since_restore: 467.0740284919739
  time_this_iter_s: 233.47639560699463
  time_total_s: 467.0740284919739
  timestamp: 1652403580
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: df4f2_00000
  
== Status ==
Current time: 2022-05-12 20:59:40 (running for 00:07:51.12)
Memory usage on this node: 14.5/31.8 GiB
Using FIFO scheduling algorithm.
Resources requested: 2.0/16 CPUs, 0/1 GPUs, 0.0/11.87 GiB heap, 0.0/5.94 GiB objects
Result logdir: C:\Users\MSI-PC\ray_results\_objective_2022-05-12_20-51-49
Nu

 pid=3448) ctive
                                               A
100%|██████████| 64/64 [07:46<00:00,  7.28s/it]A
 pid=18228) tive
                                               A
100%|██████████| 64/64 [07:47<00:00,  7.30s/it]A
2022-05-12 20:59:42,260	INFO tune.py:636 -- Total run time: 472.40 seconds (472.27 seconds for the tuning loop).


Result for _objective_df4f2_00002:
  date: 2022-05-12_20-59-42
  done: false
  epoch: 2.0
  eval_ARI: 0.044573251683269824
  eval_NMI: 0.11457223676689102
  eval_accuracy: 0.23155216284987276
  eval_loss: 2.27164626121521
  eval_runtime: 19.5697
  eval_samples_per_second: 40.164
  eval_steps_per_second: 0.409
  experiment_id: 525ded7e847e468baaa624b45e1aab6d
  hostname: XR
  iterations_since_restore: 2
  node_ip: 127.0.0.1
  objective: 0.044573251683269824
  pid: 18228
  time_since_restore: 468.2010109424591
  time_this_iter_s: 234.24334144592285
  time_total_s: 468.2010109424591
  timestamp: 1652403582
  timesteps_since_restore: 0
  training_iteration: 2
  trial_id: df4f2_00002
  
Result for _objective_df4f2_00002:
  date: 2022-05-12_20-59-42
  done: true
  epoch: 2.0
  eval_ARI: 0.044573251683269824
  eval_NMI: 0.11457223676689102
  eval_accuracy: 0.23155216284987276
  eval_loss: 2.27164626121521
  eval_runtime: 19.5697
  eval_samples_per_second: 40.164
  eval_steps_per_second: 0.409

#### 2. Clustering with Fine-tuned Electra and pre-trained Electra

In [ ]:
model_finetune = ElectraForSequenceClassification.from_pretrained(r"C:\Users\MSI-PC\Desktop\DS-GA 1012 (NLU)\Term Project\models_yahoo\run-df4f2_00001\checkpoint-64",num_labels=num_labels,output_hidden_states=True)
model_pretrain = ElectraForSequenceClassification.from_pretrained("google/electra-small-discriminator",num_labels=num_labels,output_hidden_states=True)

loading configuration file C:\Users\MSI-PC\Desktop\DS-GA 1012 (NLU)\Term Project\models_yahoo\run-df4f2_00001\checkpoint-64\config.json
Model config ElectraConfig {
  "_name_or_path": "google/electra-small-discriminator",
  "architectures": [
    "ElectraForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9"
  },
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6,
    "LABEL_7": 7,
    "LABEL_8": 8,
    "LABEL_9": 9
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "m

In [ ]:
# inputs = tokenizer("Hello, my dog is cute", max_length=64,padding="max_length",return_tensors="pt")
# outputs = model(**inputs)
# #get the text representation from the last hidden layer
# length = np.array(inputs['attention_mask'][0]).sum()
# encoding = outputs.hidden_states[-1][0].detach().numpy()[:length,:]
# encoding = encoding.mean(axis=0)

In [ ]:
true_label = test_data['category']

def text_representation(dataframe,model,tokenizer):
    representation = []
    for i in range(len(dataframe)):
        text = dataframe['text'].iloc[i]
        inputs = tokenizer(text, max_length=64,padding="max_length",truncation=True,return_tensors="pt")
        outputs = model(**inputs)
        length = np.array(inputs['attention_mask'][0]).sum()
        encoding = outputs.hidden_states[-1][0].detach().numpy()[:length,:]
        encoding = list(encoding.mean(axis=0))
        representation.append(encoding)
    return np.array(representation)

pretrain_train = text_representation(train_data,model_pretrain,tokenizer)
pretrain_test = text_representation(test_data,model_pretrain,tokenizer)
finetune_train = text_representation(train_data,model_finetune,tokenizer)
finetune_test = text_representation(test_data,model_finetune,tokenizer)

In [ ]:
pretrain_train.shape

(3928, 256)

In [ ]:
def three_metrics(true_label,preds):  
    from sklearn.metrics import accuracy_score, normalized_mutual_info_score, adjusted_rand_score
    ACC = round(accuracy_score(true_label,preds),3)
    if ACC<=1/len(true_label.unique()): #transfer the label
        keys = list(pd.value_counts(preds).index)
        values = list(pd.value_counts(true_label).index)
        dic = dict(zip(keys, values))
        preds = pd.Series(preds).map(dic)
    NMI = round(normalized_mutual_info_score(true_label,preds),3)
    ARI = round(adjusted_rand_score(true_label,preds),3)
    ACC = round(accuracy_score(true_label,preds),3)
    return {'ACC':ACC,'NMI':NMI,'ARI':ARI}

In [ ]:
#K-Means
from sklearn.cluster import KMeans
clustering_model = KMeans(n_clusters = num_labels, 
                          init = 'k-means++',
                          max_iter = 300, n_init = 10,random_state=123)
clustering_model.fit(pretrain_train)
pretrain_KMeans = clustering_model.predict(pretrain_test)
clustering_model.fit(finetune_train)
finetune_KMeans = clustering_model.predict(finetune_test)


In [ ]:
#FCM
from fcmeans import FCM
fcm = FCM(n_clusters=num_labels)
fcm.fit(np.array(pretrain_train))
pretrain_FCM = fcm.predict(pretrain_test)
fcm.fit(np.array(finetune_train))
finetune_FCM = fcm.predict(finetune_test)


In [ ]:
#LDA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
pretrain_train2 = scaler.fit_transform(pretrain_train)
pretrain_test2 = scaler.fit_transform(pretrain_test)
finetune_train2 = scaler.fit_transform(finetune_train)
finetune_test2 = scaler.fit_transform(finetune_test)

lda = LatentDirichletAllocation(n_components=num_labels, random_state=123)
lda.fit(pretrain_train2)
doc_topic_dist_unnormalized = np.matrix(lda.transform(pretrain_test2))
doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
pretrain_LDA = list(np.array(doc_topic_dist.argmax(axis=1)).T[0])
lda.fit(finetune_train2)
doc_topic_dist_unnormalized = np.matrix(lda.transform(finetune_test2))
doc_topic_dist = doc_topic_dist_unnormalized/doc_topic_dist_unnormalized.sum(axis=1)
finetune_LDA = list(np.array(doc_topic_dist.argmax(axis=1)).T[0])


In [ ]:
print('K-Means')
print('pretrained text representation:',three_metrics(true_label,pretrain_KMeans))
print('finetuned text representation:',three_metrics(true_label,finetune_KMeans))
print('FCM')
print('pretrained text representation:',three_metrics(true_label,pretrain_FCM))
print('finetuned text representation:',three_metrics(true_label,finetune_FCM))
print('LDA')
print('pretrained text representation:',three_metrics(true_label,pretrain_LDA))
print('finetuned text representation:',three_metrics(true_label,finetune_LDA))

K-Means
pretrained text representation: {'ACC': 0.101, 'NMI': 0.046, 'ARI': 0.01}
finetuned text representation: {'ACC': 0.106, 'NMI': 0.105, 'ARI': 0.046}
FCM
pretrained text representation: {'ACC': 0.105, 'NMI': 0.031, 'ARI': 0.011}
finetuned text representation: {'ACC': 0.155, 'NMI': 0.08, 'ARI': 0.042}
LDA
pretrained text representation: {'ACC': 0.128, 'NMI': 0.034, 'ARI': 0.01}
finetuned text representation: {'ACC': 0.096, 'NMI': 0.098, 'ARI': 0.041}
